In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/enron-email-dataset/emails.csv


In [4]:
import re
import string
import warnings
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline    
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, ConfusionMatrixDisplay, classification_report, accuracy_score, f1_score
from sklearn import metrics

In [5]:
df = pd.read_csv("/kaggle/input/enron-email-dataset/emails.csv", nrows = 10000)
df.describe().T

,count,unique,top,freq
file,10000,10000,allen-p/_sent_mail/1.,1
message,10000,10000,Message-ID: <18782981.1075855378110.JavaMail.e...,1


In [6]:
df = df.rename(columns={'message': 'Body'})
df.head()

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

df = df.drop(['file'], axis = 1)

df = df.copy()
def remove_first_15_lines(text):
    lines = text.split('\n')
    return '\n'.join(lines[15:])
df['Body'] = df['Body'].apply(remove_first_15_lines)


df.head(10)

,Body
0,\nJennifer:\n\nMike Rabon had just forwarded t...
1,"\nyea, i think the two dumps in the market ar..."
2,X-FileName: pallen.nsf\n\nTranswestern Pipelin...
3,\n What it's trading what I think it's reall...
4,\n Thank you for requesting additional info...
5,X-Origin: Arnold-J\nX-FileName: Jarnold.nsf\n\...
6,"\nGeorge,\n\n Your attachment is not openi..."
7,\n\n [IMAGE] [IMAGE] [IMAGE] =09\n [IMAGE] ...
8,\nhttp://messages.yahoo.com/bbs?.mm=FN&action=...
9,\n---------------------- Forwarded by Phillip ...


In [7]:
print("The First 1 Texts:",*df["Body"][:1], sep = "\n_______________________\n")

The First 1 Texts:
_______________________

Jennifer:

Mike Rabon had just forwarded to me this helpful synopsis of the opportunity 
with Kinko's as he sees it.  I would like to forward this on to James 
Thompson to assist him in targeting the right person for us to work with (and 
cc Mike Rabon).  Is this a good idea to forward on the e-mail or is it 
overkill given the e-mail I sent yesterday? Second, could I mention "Gary 
King" Mike Rabon's Kinko's contact in the e-mail?

Thanks for your suggestion!  SJ




Mr. Thompson:

Here are several "touchpoints" identified by Mike Rabon, the Enron Broadband 
Services originator, which may facilitate your finding the appropriate 
broadband solutions architect at Kinko's.  You may recall having met Mike 
Rabon in the October 6th meeting. 

Kinko's has an IP network today with access to all branches.  Kinko's had 
expressed the need to add more hubs to that network and make some changes to 
it in the spring 2001 time frame.  Enron's goal would 

In [8]:
data = df.copy()
data["No_of_Characters"] = data["Body"].apply(len)
data["No_of_Words"]=data.apply(lambda row: nltk.word_tokenize(row["Body"]), axis=1).apply(len)
data["No_of_sentence"]=data.apply(lambda row: nltk.sent_tokenize(row["Body"]), axis=1).apply(len)
data

,Body,No_of_Characters,No_of_Words,No_of_sentence
0,\nJennifer:\n\nMike Rabon had just forwarded t...,2337,435,22
1,"\nyea, i think the two dumps in the market ar...",3083,624,35
2,X-FileName: pallen.nsf\n\nTranswestern Pipelin...,417,71,2
3,\n What it's trading what I think it's reall...,974,192,7
4,\n Thank you for requesting additional info...,375,59,4
...,...,...,...,...
9995,\nyou think that's a valid excuse? whatever....,48,11,2
9996,\n\n\nHere are this week's survey results.\n\n...,1726,286,12
9997,X-bcc: \nX-Folder: \JARNOLD (Non-Privileged)\A...,417,65,1
9998,\n---------------------- Forwarded by Phillip ...,547,124,1


In [9]:
def Clean(Text):
    sms = re.sub('[^a-zA-Z]', ' ', Text) 
    sms = sms.lower() 
    sms = sms.split()
    sms = ' '.join(sms)
    return sms

data["Clean_Text"] = data["Body"].apply(Clean)
print("The First 5 Texts after cleaning:",*data["Clean_Text"][:3], sep = "\n___________________________\n")

The First 5 Texts after cleaning:
___________________________
jennifer mike rabon had just forwarded to me this helpful synopsis of the opportunity with kinko s as he sees it i would like to forward this on to james thompson to assist him in targeting the right person for us to work with and cc mike rabon is this a good idea to forward on the e mail or is it overkill given the e mail i sent yesterday second could i mention gary king mike rabon s kinko s contact in the e mail thanks for your suggestion sj mr thompson here are several touchpoints identified by mike rabon the enron broadband services originator which may facilitate your finding the appropriate broadband solutions architect at kinko s you may recall having met mike rabon in the october th meeting kinko s has an ip network today with access to all branches kinko s had expressed the need to add more hubs to that network and make some changes to it in the spring time frame enron s goal would be to make a needs assessment for 

In [10]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
No_of_Characters,10000.0,1850.3212,5971.175699,3.0,245.0,627.0,1581.0,125933.0
No_of_Words,10000.0,321.8975,864.275310,1.0,49.0,126.0,297.0,23324.0
No_of_sentence,10000.0,11.1529,25.973987,1.0,3.0,5.0,11.0,571.0


In [80]:
# Define keyword lists for each category
shopping_keywords = [
    'sale', 'discount', 'deal', 'offer', 'coupon', 'buy', 'purchase', 'shop',
    'order', 'product', 'exclusive', 'bargain', 'promo', 'special', 'free',"Sale", "Discount", "Coupon", "Deal", "Purchase", 
    "Order", "Receipt", "Shipping", "Delivery", "Product", 
    "Item", "Store", "Cart", "Checkout", "Wishlist", 
    "Bargain", "Offer", "Promotion", "Cashback", "Clearance"
]

social_keywords = [
    'friend', 'party', 'event', 'social', 'gathering', 'meet', 'hangout',
    'reunion', 'celebrate', 'birthday', 'wedding', 'invite', 'fun', 'join',"Facebook", "Twitter", "Instagram", "LinkedIn", "Friend request", 
    "Connection", "Follow", "Like", "Comment", "Share", 
    "Invite", "Event", "Group", "Meetup", "Tag", 
    "Message", "Chat", "Notification", "Timeline", "Post"
]

personal_keywords = [
    'family', 'personal', 'update', 'life', 'news', 'home', 'kids', 'school',
    'vacation', 'holiday', 'trip', 'hello', 'hi', 'hey', 'catch up',"Family", "Friend", "Dinner", "Party", "Vacation", 
    "Trip", "Photos", "Wedding", "Birthday", "Anniversary", 
    "Holiday", "Congratulations", "Invitation", "Get-together", "Celebration", 
    "Memories", "Home", "School", "Love", "Greetings"
]

job_keywords = [
    'job', 'career', 'position', 'opportunity', 'interview', 'resume',
    'application', 'hire', 'recruit', 'employment', 'work', 'office',
    'role', 'professional', 'vacancy', "Resume", "CV", "Interview", "Job offer", "Vacancy", 
    "Position", "Career", "Application", "Recruitment", "Employer", 
    "Hiring", "Opportunity", "Promotion", "Internship", "Work", 
    "Contract", "Full-time", "Part-time", "Freelance", "Job opening"
]

newsletter_keywords = [
    'newsletter', 'update', 'news', 'weekly', 'monthly', 'digest', 'subscribe',
    'issue', 'edition', 'report', 'highlights', 'bulletin', 'summary', 'email',"Subscribe", "Unsubscribe", "Update", "Edition", "Issue", 
    "Bulletin", "Digest", "Weekly", "Monthly", "Newsletter", 
    "News", "Highlights", "Trends", "Insights", "Review", 
    "Recap", "Summary", "Report", "Announcement", "Special offer","edition"
]

# Initialize category columns
data['shopping_score'] = 0
data['social_score'] = 0
data['personal_score'] = 0
data['job_score'] = 0
data['newsletter_score'] = 0

# Calculate scores based on keyword occurrences
for keyword in shopping_keywords:
    data['shopping_score'] += data['Clean_Text'].str.lower().str.count(keyword.lower())

for keyword in social_keywords:
    data['social_score'] += data['Clean_Text'].str.lower().str.count(keyword.lower())

for keyword in personal_keywords:
    data['personal_score'] += data['Clean_Text'].str.lower().str.count(keyword.lower())

for keyword in job_keywords:
    data['job_score'] += data['Clean_Text'].str.lower().str.count(keyword.lower())

for keyword in newsletter_keywords:
    data['newsletter_score'] += data['Clean_Text'].str.lower().str.count(keyword.lower())

# Assign categories based on the highest score
data['Category'] = data[['shopping_score', 'social_score', 'personal_score', 'job_score', 'newsletter_score']].idxmax(axis=1)
data['Category'] = data['Category'].str.replace('_score', '')

# Drop the score columns
data = data.drop(columns=['shopping_score', 'social_score', 'personal_score', 'job_score', 'newsletter_score'])

# Save the updated DataFrame to a CSV file
output_csv_filename = 'classified_emails.csv'
data.to_csv(output_csv_filename, index=False)

data['Category'].value_counts()

Category
personal      4502
shopping      2401
social        1568
job            805
newsletter     724
Name: count, dtype: int64

In [81]:
data.head(3)

,Body,No_of_Characters,No_of_Words,No_of_sentence,Clean_Text,Category,Tokenize_Text,Nostopword_Text,Stemmed_Text
0,\nJennifer:\n\nMike Rabon had just forwarded t...,2337,435,22,jennifer mike rabon had just forwarded to me t...,job,"[jennifer, mike, rabon, had, just, forwarded, ...","[jennifer, mike, rabon, forwarded, helpful, sy...","[jennif, mike, rabon, forward, help, synopsi, ..."
1,"\nyea, i think the two dumps in the market ar...",3083,624,35,yea i think the two dumps in the market are wh...,personal,"[yea, i, think, the, two, dumps, in, the, mark...","[yea, think, two, dumps, market, everybody, re...","[yea, think, two, dump, market, everybodi, rea..."
2,X-FileName: pallen.nsf\n\nTranswestern Pipelin...,417,71,2,x filename pallen nsf transwestern pipeline co...,social,"[x, filename, pallen, nsf, transwestern, pipel...","[x, filename, pallen, nsf, transwestern, pipel...","[x, filenam, pallen, nsf, transwestern, pipeli..."


In [86]:
data['Category'].value_counts()
column = data['Category']
df['Category'] = column

In [87]:
X = df['Body']
Y = df['Category']
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, stratify = Y ,random_state =42)

In [88]:
data["Tokenize_Text"]=data.apply(lambda row: nltk.word_tokenize(row["Clean_Text"]), axis=1)
print("The First Text after Tokenizing:",*data["Tokenize_Text"][:1], sep = "\n_______________________\n")

The First Text after Tokenizing:
_______________________
['jennifer', 'mike', 'rabon', 'had', 'just', 'forwarded', 'to', 'me', 'this', 'helpful', 'synopsis', 'of', 'the', 'opportunity', 'with', 'kinko', 's', 'as', 'he', 'sees', 'it', 'i', 'would', 'like', 'to', 'forward', 'this', 'on', 'to', 'james', 'thompson', 'to', 'assist', 'him', 'in', 'targeting', 'the', 'right', 'person', 'for', 'us', 'to', 'work', 'with', 'and', 'cc', 'mike', 'rabon', 'is', 'this', 'a', 'good', 'idea', 'to', 'forward', 'on', 'the', 'e', 'mail', 'or', 'is', 'it', 'overkill', 'given', 'the', 'e', 'mail', 'i', 'sent', 'yesterday', 'second', 'could', 'i', 'mention', 'gary', 'king', 'mike', 'rabon', 's', 'kinko', 's', 'contact', 'in', 'the', 'e', 'mail', 'thanks', 'for', 'your', 'suggestion', 'sj', 'mr', 'thompson', 'here', 'are', 'several', 'touchpoints', 'identified', 'by', 'mike', 'rabon', 'the', 'enron', 'broadband', 'services', 'originator', 'which', 'may', 'facilitate', 'your', 'finding', 'the', 'appropriate',

In [89]:
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    filtered_text = [word for word in text if word not in stop_words]
    return filtered_text

data["Nostopword_Text"] = data["Tokenize_Text"].apply(remove_stopwords)

print("The First Text after removing the stopwords: ",*data["Nostopword_Text"][:1], sep = "\n____________________________\n")

The First Text after removing the stopwords: 
____________________________
['jennifer', 'mike', 'rabon', 'forwarded', 'helpful', 'synopsis', 'opportunity', 'kinko', 'sees', 'would', 'like', 'forward', 'james', 'thompson', 'assist', 'targeting', 'right', 'person', 'us', 'work', 'cc', 'mike', 'rabon', 'good', 'idea', 'forward', 'e', 'mail', 'overkill', 'given', 'e', 'mail', 'sent', 'yesterday', 'second', 'could', 'mention', 'gary', 'king', 'mike', 'rabon', 'kinko', 'contact', 'e', 'mail', 'thanks', 'suggestion', 'sj', 'mr', 'thompson', 'several', 'touchpoints', 'identified', 'mike', 'rabon', 'enron', 'broadband', 'services', 'originator', 'may', 'facilitate', 'finding', 'appropriate', 'broadband', 'solutions', 'architect', 'kinko', 'may', 'recall', 'met', 'mike', 'rabon', 'october', 'th', 'meeting', 'kinko', 'ip', 'network', 'today', 'access', 'branches', 'kinko', 'expressed', 'need', 'add', 'hubs', 'network', 'make', 'changes', 'spring', 'time', 'frame', 'enron', 'goal', 'would', 'make'

In [90]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def stem_text(text):
    stemmed_words = [stemmer.stem(word) for word in text]
    return stemmed_words

data["Stemmed_Text"] = data["Nostopword_Text"].apply(stem_text)
print("The First Text after removing the stopwords: ",*data["Stemmed_Text"][:1], sep = "\n____________________________\n")

The First Text after removing the stopwords: 
____________________________
['jennif', 'mike', 'rabon', 'forward', 'help', 'synopsi', 'opportun', 'kinko', 'see', 'would', 'like', 'forward', 'jame', 'thompson', 'assist', 'target', 'right', 'person', 'us', 'work', 'cc', 'mike', 'rabon', 'good', 'idea', 'forward', 'e', 'mail', 'overkil', 'given', 'e', 'mail', 'sent', 'yesterday', 'second', 'could', 'mention', 'gari', 'king', 'mike', 'rabon', 'kinko', 'contact', 'e', 'mail', 'thank', 'suggest', 'sj', 'mr', 'thompson', 'sever', 'touchpoint', 'identifi', 'mike', 'rabon', 'enron', 'broadband', 'servic', 'origin', 'may', 'facilit', 'find', 'appropri', 'broadband', 'solut', 'architect', 'kinko', 'may', 'recal', 'met', 'mike', 'rabon', 'octob', 'th', 'meet', 'kinko', 'ip', 'network', 'today', 'access', 'branch', 'kinko', 'express', 'need', 'add', 'hub', 'network', 'make', 'chang', 'spring', 'time', 'frame', 'enron', 'goal', 'would', 'make', 'need', 'assess', 'desir', 'structur', 'kinko', 'would',

In [91]:
corpus = data["Stemmed_Text"].apply(lambda x: ' '.join(x)).tolist()
corpus[:5]
print("The First 5 lines in corpus ",*corpus[:5], sep = "\n___________________________\n")
print(type(corpus))

The First 5 lines in corpus 
___________________________
jennif mike rabon forward help synopsi opportun kinko see would like forward jame thompson assist target right person us work cc mike rabon good idea forward e mail overkil given e mail sent yesterday second could mention gari king mike rabon kinko contact e mail thank suggest sj mr thompson sever touchpoint identifi mike rabon enron broadband servic origin may facilit find appropri broadband solut architect kinko may recal met mike rabon octob th meet kinko ip network today access branch kinko express need add hub network make chang spring time frame enron goal would make need assess desir structur kinko would like see pertain near term network plan ip transport capac requir mbp burst potenti ip transit capac requir current contract usag internet transit kinko com particular high potenti ip transit capac storag hub branch number grow may need manag storag enron solut reduc elimin need buy storag devic mike rabon would like speak

In [92]:
tfidf = TfidfVectorizer()
empty_array = np.empty((10000, 32355))

X = tfidf.fit_transform(corpus).toarray()
#Let's have a look at our feature 
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [93]:
label_encoder = LabelEncoder()
df["Category"] = label_encoder.fit_transform(df["Category"])
df['Category'].value_counts()

Category
2    4502
3    2401
4    1568
0     805
1     724
Name: count, dtype: int64

In [94]:
Y = df['Category']
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, stratify = Y, test_size = 0.3, random_state = 42)

In [95]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

In [96]:
# defining the model
model = Sequential()

# adding input layer
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))

# adding hidden layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(Y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [97]:
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=20, batch_size=32, verbose=1)

Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - accuracy: 0.4458 - loss: 1.3911 - val_accuracy: 0.7287 - val_loss: 0.7407
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.7891 - loss: 0.6181 - val_accuracy: 0.8653 - val_loss: 0.4776
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.9194 - loss: 0.2670 - val_accuracy: 0.8807 - val_loss: 0.4808
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - accuracy: 0.9538 - loss: 0.1700 - val_accuracy: 0.8970 - val_loss: 0.4611
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.9636 - loss: 0.1204 - val_accuracy: 0.8980 - val_loss: 0.5148
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - accuracy: 0.9774 - loss: 0.0833 - val_accuracy: 0.8910 - val_loss: 0.5663
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - accuracy: 0.9820 - loss: 0.0748 - val_accuracy: 0.8940 - val_loss: 0.6056
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.9836 - loss: 0.0660 - val_acc

In [98]:
loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
print(f'Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')

Loss: 1.0419, Accuracy: 0.8950


In [99]:
Y_pred = model.predict(X_test)
Y_pred_classes = np.argmax(Y_pred, axis=1)
Y_test_classes = np.argmax(Y_test, axis=1)

print(f'Predicted classes: {Y_pred_classes[:100]}')
print(f'True classes: {Y_test_classes[:100]}')

94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Predicted classes: [2 2 0 0 3 4 2 2 2 2 1 4 2 2 2 2 2 4 2 0 2 2 2 2 2 2 2 2 2 0 1 2 2 2 2 3 3
 2 3 3 3 4 2 0 2 0 3 2 2 2 2 4 2 1 2 0 3 4 3 2 4 3 2 0 3 3 2 0 2 2 4 2 3 2
 2 1 2 2 2 2 2 1 2 4 2 3 2 0 1 2 3 2 1 2 2 3 4 3 2 3]
True classes: [2 2 0 0 3 4 2 2 2 2 1 4 2 2 2 2 2 4 2 0 2 2 2 2 2 2 2 2 2 0 1 2 2 2 2 3 3
 2 3 3 3 4 2 0 2 2 3 2 2 3 3 4 2 1 2 0 3 4 3 2 4 3 2 0 3 3 0 0 2 2 4 2 3 2
 2 1 2 2 3 2 2 1 2 4 2 3 2 0 1 2 3 2 1 2 2 3 4 3 2 3]


In [104]:
text = "this is the upgraded edition of DailyDoseOfInternet. Happy reading!"
def predict_category(text):
    text = Clean(text)
    text = nltk.word_tokenize(text)
    text = remove_stopwords(text)
    text = stem_text(text)

    msg = ' '.join(text)
    x = tfidf.transform([msg]).toarray()  # Use transform instead of fit_transform
    y_pred = model.predict(x)
    y_pred_class = np.argmax(y_pred, axis=1)

    return label_encoder.inverse_transform(y_pred_class)[0]

predicted_category = predict_category(text)
print(f'Predicted category: {predicted_category}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted category: newsletter


In [74]:
model.save('model.h5')